# Challenge: timerand

In [3]:
import requests
def get_challnge(challenge:str) -> str:
    server = "https://ciberseguridad.diplomatura.unc.edu.ar/cripto"
    email = "alvarmaciel@gmail.com"
    response =  requests.get(f"{server}/{challenge}/{email}/challenge")
    return response.text

In [30]:
raw_challenge = get_challnge("timerand")
raw_challenge

'From: User <user@example.com>\nDate: Fri Jan 14 19:45:58 UTC 2022\nTo: alvarmaciel@gmail.com\n\nJKjGe49INxspqtMMtm6hesPCxag4ljpDzLhlmyagYkEpcuzz7cSdMYMaigA1Rqkwqk+KtVxODwNV8vGM2tUsWgTJ4xA4RFrx0Ww2o/o27ocLKUwfQiVPENw198AI25iAljOAmOioNJd/IRmzLzEW88RknUjRF4udo+0YlNeOtwGWuUR4iMj7+otL1XnymNFMxkXzq+PeKloTS9SbmzKDVQr8HG9BjE5Fd/emVJQ+y9K/qDU637IfIhOGjeKX/k03f9xMt00UWTi852J2M+aF1xyOAmKTADSzlEDU1jWfD1Czy8TFD0VJnnImtSjOpThocCjwnJTKxQhjFaiPJjqPAb/jSNhpkWNFebPUrXeRDOAIKYGegZfekvITvyU0RFziAJEVUv1LmUEOyoKzlFCv1OYEAJNn70qNzYvhBHE1L+IrubFzCl4o7xeiJlN2sIVdCaaGENfV3gMgbGPtAtlbOczHWN9Gz1Sq4oFBdNWgF58Z5Q8vkoMKG5o0WQJqduohZG5KrDp9M3jjSbvH8ZhoBg==\n'

# Ejemplo Generador de codigo

In [15]:
import time
import base64
from hashlib import md5
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import rsa, padding as asymmetric_padding
from cryptography.hazmat.primitives import serialization, hashes, padding as symmetric_padding
from cryptography.hazmat.backends import default_backend  

def generate_challenge(message, public_key):
    # Obtenemos el tiempo en microsegundos
    now = int(time.time()*1000000)
    # Este es un mal mecanismo para generar una clave
    key_seed = now.to_bytes(8,"big")
    key = md5(key_seed).digest()
    # Este es un mal mecanismo para generar un IV
    iv_seed = (now+1).to_bytes(8,"big")
    iv = md5(iv_seed).digest()
    # Ciframos el mensaje con AES-128 usando la clave `key`
    # Obtenemos una instancia del cifrador
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    # Usamos PKCS7 como mecanismo de relleno
    padder = symmetric_padding.PKCS7(128).padder()
    padded = padder.update(message) + padder.finalize()
    # Ciframos
    ciphertext = encryptor.update(padded) + encryptor.finalize()
    # Ciframos la clave secreta con la clave pública del destinatario
    encrypted_key = public_key.encrypt(
        key,
        asymmetric_padding.OAEP(
            mgf=asymmetric_padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    assert(len(encrypted_key) == 128)
    # El desafío está compuesto por la clave cifrada, el iv y el texto cifrado
    return base64.b64encode(encrypted_key + iv + ciphertext)

# Clave pública serializada
pem = b'-----BEGIN PUBLIC KEY-----\nMIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQDIgCvlphhRoOt39+rBUxdqrivb\nxVdYmlGXkT/5mO6/fwYzA/yXeg1aKEIj1f1l1yXWecN0iYrDUflwyN5aMj6nH+zW\nYfMOVhp2seFya10vXA24UX5o5Ei7JLcnf1ubO3VfH4Fttcvvy27Qmkii3uedOTFi\nZtZn/U1dhm1rYhtrPwIDAQAB\n-----END PUBLIC KEY-----\n'
message = b"Este es el texto claro del mensaje"
# Deserializamos la clave pública
public_key = serialization.load_pem_public_key(pem)

generate_challenge(message, public_key)

b'hZ4LN4T8J5T7RWkWz4Bhu51ybciOxCQeuxzxC8FdWVEI7H+SGJnDcEIPNZmSRtVwmTNKZEIlf1tZfA6RP5QgzAnxps5MjvSuTSzMFSvuRMUyuUMu0TT59gWVlZE5oLFcro+ST6Lj7E6BagFCDAZAU9jlwiKaY8SPfgKRHRGfNuAwyK2dBUxwVX6GTqnQsy+B9YP2QUlC9BemKFElmJZXWt2JnvMLoATxScNp3B8kR/2Qz/3H3V76anNZ5S8jGk5h'

In [35]:
# Timestamp helper
from datetime import datetime
def datetime_to_timestamp(date: datetime) -> float:
    return date.timestamp()

In [40]:
print(raw_challenge)

From: User <user@example.com>
Date: Fri Jan 14 19:45:58 UTC 2022
To: alvarmaciel@gmail.com

JKjGe49INxspqtMMtm6hesPCxag4ljpDzLhlmyagYkEpcuzz7cSdMYMaigA1Rqkwqk+KtVxODwNV8vGM2tUsWgTJ4xA4RFrx0Ww2o/o27ocLKUwfQiVPENw198AI25iAljOAmOioNJd/IRmzLzEW88RknUjRF4udo+0YlNeOtwGWuUR4iMj7+otL1XnymNFMxkXzq+PeKloTS9SbmzKDVQr8HG9BjE5Fd/emVJQ+y9K/qDU637IfIhOGjeKX/k03f9xMt00UWTi852J2M+aF1xyOAmKTADSzlEDU1jWfD1Czy8TFD0VJnnImtSjOpThocCjwnJTKxQhjFaiPJjqPAb/jSNhpkWNFebPUrXeRDOAIKYGegZfekvITvyU0RFziAJEVUv1LmUEOyoKzlFCv1OYEAJNn70qNzYvhBHE1L+IrubFzCl4o7xeiJlN2sIVdCaaGENfV3gMgbGPtAtlbOczHWN9Gz1Sq4oFBdNWgF58Z5Q8vkoMKG5o0WQJqduohZG5KrDp9M3jjSbvH8ZhoBg==



In [38]:
b64_message = raw_challenge.split("\n")[4]
b64_message

'JKjGe49INxspqtMMtm6hesPCxag4ljpDzLhlmyagYkEpcuzz7cSdMYMaigA1Rqkwqk+KtVxODwNV8vGM2tUsWgTJ4xA4RFrx0Ww2o/o27ocLKUwfQiVPENw198AI25iAljOAmOioNJd/IRmzLzEW88RknUjRF4udo+0YlNeOtwGWuUR4iMj7+otL1XnymNFMxkXzq+PeKloTS9SbmzKDVQr8HG9BjE5Fd/emVJQ+y9K/qDU637IfIhOGjeKX/k03f9xMt00UWTi852J2M+aF1xyOAmKTADSzlEDU1jWfD1Czy8TFD0VJnnImtSjOpThocCjwnJTKxQhjFaiPJjqPAb/jSNhpkWNFebPUrXeRDOAIKYGegZfekvITvyU0RFziAJEVUv1LmUEOyoKzlFCv1OYEAJNn70qNzYvhBHE1L+IrubFzCl4o7xeiJlN2sIVdCaaGENfV3gMgbGPtAtlbOczHWN9Gz1Sq4oFBdNWgF58Z5Q8vkoMKG5o0WQJqduohZG5KrDp9M3jjSbvH8ZhoBg=='

In [39]:
date = datetime_to_timestamp(datetime(2022,1,14,19,45,58))
date

1642200358.0

In [41]:
b_message = base64.b64decode(b64_message)

In [44]:

print("IV:", iv.hex())

IV: 96b9447888c8fbfa8b4bd579f298d14c


In [49]:
def decipher(b64_decoded_message: bytes, timestamp:float):
    # decode
    rsa_encrypted_key = b64_decoded_message[:128]
    iv = b64_decoded_message[128:144]
    cipher_text=b64_decoded_message[144:]
    # key range
    starter_key=int(timestamp * 1_000_000)
    for t in range(starter_key, starter_key + 1_000_000):
        # Try to decode
        timestamp_candidate_bytes = t.to_bytes(8,"big")
        timestamp_candidate_md5 = md5(timestamp_candidate_bytes).digest()
        candidate_cipher = Cipher(algorithms.AES(timestamp_candidate_md5), modes.CBC(iv), backend=default_backend())
        decryptor = candidate_cipher.decryptor()
        candidate = decryptor.update(cipher_text) + decryptor.finalize()
        # verify
    return result
    
    